# TCR分析

目前celescope支持cdr3，cellranger，trust4三种vdj分析（以cellranger为例）。

- sample：生成样本信息
- barcode：解复用barcode，主要过滤fastq文件中不包含barcode的reads和低质量的reads
- convert：将barcode和umi序列转换成软件需要的格式
- assemble：T/BCR组装
- summarize：将barcode和umi序列转换回来，生成主要的结果文件，组装结果的VDJ注释信息，序列以及clonotypes结果
- match：将组装出的细胞联合转录组进行分析
- mapping：组装出的细胞映射回转录组中的UMAP-plot可视化，统计组装效率等指标

1.下载软件和vdj reference

wget -O cellranger-6.1.2.tar.gz "https://cf.10xgenomics.com/releases/cell-vdj/cellranger-6.1.2.tar.gz?Expires=1646072261&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZi4xMHhnZW5vbWljcy5jb20vcmVsZWFzZXMvY2VsbC12ZGovY2VsbHJhbmdlci02LjEuMi50YXIuZ3oiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE2NDYwNzIyNjF9fX1dfQ__&Signature=Z-2m906CV5Rb1snIAga-QDSXYSZ8cNqCj1EECGP4uloU3qH~uCMH42MHf4TNnDL2zAsKA7cXsCsQYz0A9yJdNh7dfRT8ohpuAzASFx5Pj-bkqfw4p2tql55IIaPN0zqxyUuyZ9sfKl5qTQX82LoVolRpiBUL8dF9nr~bA2P1gJZ~xg1QssS7icR5MmTzvKKS5NYkezG8vWaTiEdXU0nuKI2ciZSX5GOMeIRW-YYR7mJwHmBbTVxe0o-uBuUtqor0Y98jdIv8Z~dwMjujRjrEShdCGNixTSonGzeS2~9CXqWquCJIOolqFFkcFHgXkD7ZWNfSXWbTxuF57rCsub98pA__&Key-Pair-Id=APKAI7S6A5RYOXBWRPDA"

Reference: human and mouse

wget https://cf.10xgenomics.com/supp/cell-vdj/refdata-cellranger-vdj-GRCh38-alts-ensembl-5.0.0.tar.gz

wget https://cf.10xgenomics.com/supp/cell-vdj/refdata-cellranger-vdj-GRCm38-alts-ensembl-5.0.0.tar.gz

tar -xzvf cellranger-6.1.2.tar.gz

tar -xzvf refdata-cellranger-vdj-GRCh38-alts-ensembl-5.0.0.tar.gz

tar -xzvf refdata-cellranger-vdj-GRCm38-alts-ensembl-5.0.0.tar.gz

2.创建mapfile文件，multi_flv_CR命令生成shell文件

multi_flv_CR\

  --mapfile ./flv.mapfile\

  --seqtype TCR\

  --ref_path "/SGRNJ/Database/script/soft/cellranger/vdj_ref/6.0.0/hs/refdata-cellranger-vdj-GRCh38-alts-ensembl-5.0.0"\

  --soft_path "/SGRNJ03/randd/cjj/soft/cellranger/cellranger-6.1.2/cellranger"\

  --mod shell

- cat flv.mapfile(第四列为该样本对应的转录组分析路径)

fltest&emsp;../../celescope_test_data/flv/&emsp;test1&emsp;../../celescope_test_data/flv/fake_match_dir

3.运行生成的shell文件

sh shell/test1.sh

celescope flv_CR sample --outdir .//test1/00.sample --sample test1 --thread 8 --chemistry flv  --fq1 /SGRNJ06/randd/USER/cjj/TESTDATA/test_cellranger/testdata/BCRdata/test1_1.fq.gz

celescope flv_CR barcode --outdir .//test1/01.barcode --sample test1 --thread 8 --chemistry flv --lowNum 2 --allowNoLinker  --fq1 /SGRNJ06/randd/USER/cjj/TESTDATA/test_cellranger/testdata/BCRdata/test1_1.fq.gz --fq2 /SGRNJ06/randd/USER/cjj/TESTDATA/test_cellranger/testdata/BCRdata/test1_2.fq.gz

celescope flv_CR convert --outdir .//test1/02.convert --sample test1 --thread 8 --soft_path "/SGRNJ06/randd/USER/cjj/soft/cellranger-7.0.0/cellranger" --tenX_chemistry V2  --fq2 .//test1/01.barcode/test1_2.fq

celescope flv_CR assemble --outdir .//test1/03.assemble --sample test1 --thread 8 --ref_path "/SGRNJ06/randd/USER/cjj/soft/refdata-cellranger-vdj-GRCm38-alts-ensembl-7.0.0" --soft_path "/SGRNJ06/randd/USER/cjj/soft/cellranger-7.0.0/cellranger" --mem 8 --seqtype BCR  --fqs_dir .//test1/02.convert

celescope flv_CR summarize --outdir .//test1/04.summarize --sample test1 --thread 8 --seqtype BCR --soft_path "/SGRNJ06/randd/USER/cjj/soft/cellranger-7.0.0/cellranger"  --barcode_convert_json .//test1/02.convert/barcode_convert.json --assemble_out .//test1/03.assemble/test1/outs

celescope flv_CR match --outdir .//test1/05.match --sample test1 --thread 8 --seqtype BCR  --match_dir /SGRNJ03/randd/RD20040201_SCOPEv2_TCR/20220412/M_0330splN1_NLib/ --summarize_out .//test1/04.summarize

celescope flv_CR mapping --outdir .//test1/06.mapping --sample test1 --thread 8 --seqtype BCR  --match_dir /SGRNJ03/randd/RD20040201_SCOPEv2_TCR/20220412/M_0330splN1_NLib/ --match_out .//test1/05.match

# reference
- test_script:

https://github.com/singleron-RD/celescope_test_script
- test_data:

https://github.com/singleron-RD/celescope_test_data